In [1]:
# ANÁLISIS FORENSE DEL MODELO - BUSCANDO PROBLEMAS
print("🕵️ ANÁLISIS FORENSE - BÚSQUEDA DE ERRORES CRÍTICOS")
print("=" * 60)

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import joblib

# Cargar todos los datasets
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_val = np.load('X_validation.npy')
y_val = np.load('y_validation.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')
encoder = joblib.load('encoder.pkl')

print("📊 VERIFICACIÓN BÁSICA DE DATOS:")
print(f"   Train: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
print(f"   Val: {X_val.shape[0]:,} samples, {X_val.shape[1]} features")
print(f"   Test: {X_test.shape[0]:,} samples, {X_test.shape[1]} features")

# 🚨 PRUEBA 1: ¿HAY DUPLICADOS ENTRE SETS?
print(f"\n🔍 PRUEBA 1 - BUSCAR DUPLICADOS ENTRE SETS:")

# Convertir a strings para comparación
X_train_str = [str(row) for row in X_train]
X_val_str = [str(row) for row in X_val]
X_test_str = [str(row) for row in X_test]

train_val_overlap = len(set(X_train_str) & set(X_val_str))
train_test_overlap = len(set(X_train_str) & set(X_test_str))
val_test_overlap = len(set(X_val_str) & set(X_test_str))

print(f"   Train-Val overlap: {train_val_overlap} samples")
print(f"   Train-Test overlap: {train_test_overlap} samples")
print(f"   Val-Test overlap: {val_test_overlap} samples")

if train_val_overlap > 0 or train_test_overlap > 0 or val_test_overlap > 0:
    print(f"   🚨 PROBLEMA DETECTADO: Data leakage entre sets!")
else:
    print(f"   ✅ Sin duplicados exactos entre sets")


🕵️ ANÁLISIS FORENSE - BÚSQUEDA DE ERRORES CRÍTICOS
📊 VERIFICACIÓN BÁSICA DE DATOS:
   Train: 7,988 samples, 19 features
   Val: 1,707 samples, 19 features
   Test: 1,711 samples, 19 features

🔍 PRUEBA 1 - BUSCAR DUPLICADOS ENTRE SETS:
   Train-Val overlap: 949 samples
   Train-Test overlap: 962 samples
   Val-Test overlap: 221 samples
   🚨 PROBLEMA DETECTADO: Data leakage entre sets!


In [2]:
# PRUEBA 2: ANALIZAR FEATURES SOSPECHOSAS
print(f"\n🔍 PRUEBA 2 - ANÁLISIS DE FEATURES:")

# ¿Hay features con varianza 0? (constantes)
feature_vars = np.var(X_train, axis=0)
zero_var_features = np.sum(feature_vars == 0)
print(f"   Features con varianza 0: {zero_var_features}")

# ¿Hay features con correlación perfecta con el target?
from scipy.stats import pearsonr

print(f"   Correlación features vs target (top 5):")
correlations = []
for i in range(X_train.shape[1]):
    corr, _ = pearsonr(X_train[:, i], y_train)
    correlations.append((i, abs(corr)))

correlations.sort(key=lambda x: x[1], reverse=True)
for i, (feat_idx, corr) in enumerate(correlations[:5]):
    print(f"      Feature {feat_idx}: r={corr:.3f}")
    if corr > 0.95:
        print(f"         🚨 SOSPECHOSO: Correlación muy alta!")

# PRUEBA 3: DISTRIBUCION DE CLASES
print(f"\n🔍 PRUEBA 3 - DISTRIBUCIÓN DE CLASES:")

def analyze_class_distribution(y, name):
    unique, counts = np.unique(y, return_counts=True)
    print(f"   {name}:")
    for cls, count in zip(unique, counts):
        class_name = encoder.classes_[cls]
        pct = count / len(y) * 100
        print(f"      {class_name}: {count} ({pct:.1f}%)")

analyze_class_distribution(y_train, "Train")
analyze_class_distribution(y_val, "Validation")
analyze_class_distribution(y_test, "Test")



🔍 PRUEBA 2 - ANÁLISIS DE FEATURES:
   Features con varianza 0: 0
   Correlación features vs target (top 5):
      Feature 0: r=0.646
      Feature 6: r=0.385
      Feature 12: r=0.244
      Feature 11: r=0.175
      Feature 13: r=0.160

🔍 PRUEBA 3 - DISTRIBUCIÓN DE CLASES:
   Train:
      caminar_hacia: 1725 (21.6%)
      caminar_regreso: 1659 (20.8%)
      girar: 1595 (20.0%)
      ponerse_pie: 1560 (19.5%)
      sentarse: 1449 (18.1%)
   Validation:
      caminar_hacia: 369 (21.6%)
      caminar_regreso: 354 (20.7%)
      girar: 341 (20.0%)
      ponerse_pie: 333 (19.5%)
      sentarse: 310 (18.2%)
   Test:
      caminar_hacia: 370 (21.6%)
      caminar_regreso: 355 (20.7%)
      girar: 342 (20.0%)
      ponerse_pie: 334 (19.5%)
      sentarse: 310 (18.1%)


In [3]:
# PRUEBA 4: TEST DE OVERFITTING EXTREMO
print(f"\n🔍 PRUEBA 4 - DETECTAR OVERFITTING SEVERO:")

# Crear modelo completamente nuevo desde cero
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Modelo con arquitectura diferente
mlp_test = MLPClassifier(
    hidden_layer_sizes=(50,),  # Más simple
    random_state=999,  # Seed diferente
    max_iter=500,
    alpha=0.01  # Regularización
)

print(f"   Entrenando modelo nuevo con arquitectura diferente...")
mlp_test.fit(X_train, y_train)

# Evaluar
train_acc_new = accuracy_score(y_train, mlp_test.predict(X_train))
val_acc_new = accuracy_score(y_val, mlp_test.predict(X_val))
test_acc_new = accuracy_score(y_test, mlp_test.predict(X_test))

print(f"   NUEVO MODELO - Arquitectura más simple:")
print(f"      Train: {train_acc_new:.3f}")
print(f"      Val: {val_acc_new:.3f}")
print(f"      Test: {test_acc_new:.3f}")

# Cross-validation en datos originales
cv_scores = cross_val_score(mlp_test, X_train, y_train, cv=5)
print(f"   CV Score: {cv_scores.mean():.3f} (±{cv_scores.std():.3f})")

if test_acc_new >= 0.95:
    print(f"   🤔 SOSPECHOSO: Incluso modelo más simple da >95%")
else:
    print(f"   ✅ Modelo simple da accuracy más realista")



🔍 PRUEBA 4 - DETECTAR OVERFITTING SEVERO:
   Entrenando modelo nuevo con arquitectura diferente...
   NUEVO MODELO - Arquitectura más simple:
      Train: 1.000
      Val: 0.998
      Test: 1.000
   CV Score: 0.998 (±0.001)
   🤔 SOSPECHOSO: Incluso modelo más simple da >95%


In [4]:
# DIAGNÓSTICO FINAL Y RECOMENDACIONES
print(f"\n🎯 DIAGNÓSTICO FINAL:")
print("=" * 30)

issues_found = []

# Verificar cada problema potencial
if train_val_overlap > 0 or train_test_overlap > 0:
    issues_found.append("Data leakage entre datasets")

if zero_var_features > 0:
    issues_found.append(f"{zero_var_features} features constantes")

if any(corr > 0.95 for _, corr in correlations[:3]):
    issues_found.append("Features con correlación sospechosa")

if test_acc_new >= 0.98:
    issues_found.append("Modelo simple también da accuracy muy alta")

print(f"🔍 PROBLEMAS DETECTADOS:")
if issues_found:
    for issue in issues_found:
        print(f"   🚨 {issue}")
else:
    print(f"   ✅ Sin problemas evidentes detectados")

print(f"\n💡 INTERPRETACIÓN:")
if len(issues_found) == 0:
    print(f"   🎯 El modelo parece legítimo:")
    print(f"   • Sin duplicados entre datasets")
    print(f"   • Features con varianza normal")
    print(f"   • Sin correlaciones perfectas sospechosas")
    print(f"   • Problema realmente puede ser tan distintivo")

    verdict = "MODELO PROBABLEMENTE LEGÍTIMO"
    recommendation = "Proceder con deployment, añadir monitoreo robusto"

elif len(issues_found) >= 2:
    print(f"   🚨 Múltiples señales de alarma:")
    print(f"   • Posible data leakage o error en pipeline")
    print(f"   • Revisar proceso de preparación de datos")

    verdict = "MODELO SOSPECHOSO"
    recommendation = "REVISAR PIPELINE DE DATOS ANTES DE DEPLOYMENT"

else:
    print(f"   📊 Señales mixtas:")
    print(f"   • Algunos indicadores preocupantes")
    print(f"   • Pero puede ser dataset genuinamente fácil")

    verdict = "PRECAUCIÓN RECOMENDADA"
    recommendation = "Deployment con monitoreo intensivo inicial"

print(f"\n🏆 VEREDICTO: {verdict}")
print(f"🎯 RECOMENDACIÓN: {recommendation}")



🎯 DIAGNÓSTICO FINAL:
🔍 PROBLEMAS DETECTADOS:
   🚨 Data leakage entre datasets
   🚨 Modelo simple también da accuracy muy alta

💡 INTERPRETACIÓN:
   🚨 Múltiples señales de alarma:
   • Posible data leakage o error en pipeline
   • Revisar proceso de preparación de datos

🏆 VEREDICTO: MODELO SOSPECHOSO
🎯 RECOMENDACIÓN: REVISAR PIPELINE DE DATOS ANTES DE DEPLOYMENT
